**note** = in last cell group give folder of images to test and get accuracy and predicted result

for test you can put images in test folder and give path to last cell.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
#here path1 is train dataset and path2 is test dataset
path1 = '/content/drive/MyDrive/dataset 0s and 1s/train'
path2= '/content/drive/MyDrive/dataset 0s and 1s/test'

In [28]:
import os
import cv2

#function to rename image name in format of digit_0(index).png , digit_1(index).png

for test images in my code need to have specific format as shown to use.because i am taking true labels from name of file.

In [29]:
def rename_images(folder_path):

  for index, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
      old_path = os.path.join(folder_path, filename)
      new_filename = f"digit_1({index}).png"
      new_path = os.path.join(folder_path, new_filename)
      os.rename(old_path, new_path)

# Convert an image to binary code based on a threshold

 here converting image to (3,3)size and converting it in binary code for that i chose threshould value = 250  after experperimenting with values.

 if pixals intensity > threshould then code = 0.

 if pixals intensity < threshould then code = 1.

In [30]:
def get_binary_code(img, threshold=250):

    img_resized = cv2.resize(img, (3, 3))
    binary_values = [1 if pixel < threshold else 0 for pixel in img_resized.flatten()]
    return ''.join(map(str, binary_values))


#Convert a binary string to hexadecimal.

for simplicity i am converting binary string into hexadecimal value

In [31]:
def binary_to_hex(binary_code):

    decimal_value = int(binary_code, 2)
    return hex(decimal_value)

#mathematical approch

so after converting into hexadecimal we get for each img one hexadecimal number.

to predict number we need calculate P(1/I) and P(0/I) and compare both.

if P(1/I) > P(0/I) then the image is 1.

if P(1/I) < P(0/I) then the image is 0.

so we will take random img then find hexadecimal code and check that code in both 0s and 1s codes  and find P(1/I) , P(0/I) and predict number.


#Compute the frequency of hexadecimal codes for images labeled as 0 and 1.

here we are calculating counts of each hexadecimal in 0s and 1s

In [33]:
def compute_hex_code_counts(image_folder, threshold=250):

    hex_code_counts_0 = {}
    hex_code_counts_1 = {}

    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)

        # Read and process the image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        binary_code = get_binary_code(img, threshold)
        hex_code = binary_to_hex(binary_code)

        # Extract the true label
        if 'digit_0' in img_name:
            hex_code_counts_0[hex_code] = hex_code_counts_0.get(hex_code, 0) + 1
        elif 'digit_1' in img_name:
            hex_code_counts_1[hex_code] = hex_code_counts_1.get(hex_code, 0) + 1

    return hex_code_counts_0, hex_code_counts_1

#Predict the label of a test image based on hexadecimal code frequencies.
finding  counts within a  (± range) and choosing  the one with more counts.

in this function we will find binary code of trst img

then find count of that hex in 0s and 1s

we will check in range :± range

then comare the counts and five prediction

if both counts are same default prediction is 1

In [35]:
def predict_image(test_image_path, hex_code_counts_0, hex_code_counts_1, threshold=250,range_=2):

    test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
    binary_code = get_binary_code(test_image, threshold)
    test_hex_code = binary_to_hex(binary_code)

    # counts for 0 and 1
    count_0 = hex_code_counts_0.get(test_hex_code, 0)
    count_1 = hex_code_counts_1.get(test_hex_code, 0)

    # Print counts
    print(f"Hex code: {test_hex_code}, Count for 0: {count_0}, Count for 1: {count_1}")


    count_0_in_range = count_0 + range_
    count_1_in_range = count_1 + range_

    # Compare the counts within the range and predict.
    if count_0_in_range > count_1_in_range:
        return 0
    elif count_1_in_range > count_0_in_range:
        return 1
    else:
        return 1

In [37]:
#example on 1 img

compute_hex_code_counts(path1, threshold=250)
predict_image('/content/drive/MyDrive/dataset 0s and 1s/test1 original1s/digit_1(256).png', hex_code_counts_0, hex_code_counts_1, threshold=250,range_=2)

#here we get 1 as answer and count of 0 and 1.

Hex code: 0x90, Count for 0: 4, Count for 1: 57


1

#Predict labels for all images in the test folder and calculate accuracy.

creating function to get prediction from folder and accuracy for easy calculations

In [39]:
def predict_images_in_folder(test_folder, hex_code_counts_0, hex_code_counts_1, threshold=250, range_=5):

    predictions = {}
    correct_predictions = 0
    total_images = 0

    for img_name in os.listdir(test_folder):
        img_path = os.path.join(test_folder, img_name)

        # Extract the true label
        if 'digit_0' in img_name:
            true_label = 0
        elif 'digit_1' in img_name:
            true_label = 1
        else:
            continue

        # Predict the label
        predicted_label = predict_image(img_path, hex_code_counts_0, hex_code_counts_1, threshold, range_ =5)
        predictions[img_name] = predicted_label


        if predicted_label == true_label:
            correct_predictions += 1

        total_images += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_images if total_images > 0 else 0
    return predictions, accuracy

# Predict labels for test images and calculate accuracy

here  in test folder i gave approx 100 imgs and got 83.33
% accuracy

In [40]:
# Compute hexadecimal code counts for the train images

hex_code_counts_0, hex_code_counts_1 = compute_hex_code_counts(path1)

In [42]:

# Predict labels for test images and calculate accuracy
predictions, accuracy = predict_images_in_folder('/content/drive/MyDrive/dataset 0s and 1s/test original mix', hex_code_counts_0, hex_code_counts_1, range_=5)

# Print predictions and accuracy
for img_name, prediction in predictions.items():
    print(f"Image: {img_name}, Predicted digit: {prediction}")
print(f"Accuracy: {accuracy * 100:.2f}%")


#here  in test folder i gave 100 imgs and got 83% accuracy


Hex code: 0x82, Count for 0: 11, Count for 1: 4
Hex code: 0x80, Count for 0: 56, Count for 1: 43
Hex code: 0x48, Count for 0: 5, Count for 1: 2
Hex code: 0x22, Count for 0: 11, Count for 1: 0
Hex code: 0xaa, Count for 0: 6, Count for 1: 0
Hex code: 0x29, Count for 0: 1, Count for 1: 0
Hex code: 0x2, Count for 0: 41, Count for 1: 23
Hex code: 0x0, Count for 0: 98, Count for 1: 142
Hex code: 0x90, Count for 0: 4, Count for 1: 57
Hex code: 0x49, Count for 0: 0, Count for 1: 3
Hex code: 0x102, Count for 0: 1, Count for 1: 1
Hex code: 0x80, Count for 0: 56, Count for 1: 43
Hex code: 0xa2, Count for 0: 15, Count for 1: 1
Hex code: 0xa2, Count for 0: 15, Count for 1: 1
Hex code: 0x22, Count for 0: 11, Count for 1: 0
Hex code: 0x82, Count for 0: 11, Count for 1: 4
Hex code: 0x22, Count for 0: 11, Count for 1: 0
Hex code: 0x2e, Count for 0: 1, Count for 1: 0
Hex code: 0x0, Count for 0: 98, Count for 1: 142
Hex code: 0x0, Count for 0: 98, Count for 1: 142
Hex code: 0x12, Count for 0: 0, Count fo